実際に動いたコードしか突っ込まないこと！！！
（ここでエラーが出ると調整が大変です。)

In [144]:
import sys
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
from glob import glob
from pathlib import Path
import joblib
import pickle
import itertools
from tqdm.auto import tqdm
import torch
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GroupKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, roc_auc_score, matthews_corrcoef, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import category_encoders as ce
import lightgbm as lgb
#import optuna.integration.lightgbm as lgb
import xgboost as xgb
from catboost import Pool, CatBoostRegressor, CatBoostClassifier
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras import backend as K
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
import torch.optim as optim
import tensorflow as tf
from keras.models import load_model
from keras.callbacks import Callback
from keras.models import clone_model

pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',100)

# ====================================================
# Configurations
# ====================================================
class CFG:
    VER = 5.2
    AUTHOR = 'naokisusami'
    COMPETITION = 'FDUA2'
    DATA_PATH = Path('/data')
    OOF_DATA_PATH = Path('/oof')
    MODEL_DATA_PATH = Path('/models')
    SUB_DATA_PATH = Path('/submission')
    METHOD_LIST = [ 'neuralnetwork', 'adaboost','lightgbm', 'xgboost', 'catboost']
    seed = 42
    n_folds = 7
    target_col = 'MIS_Status'
    metric = 'f1_score'
    metric_maximize_flag = True
    num_boost_round = 500
    early_stopping_round = 200
    verbose = 25
    classification_lgb_params = {
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.05,
        'seed': seed,
    }
    classification_xgb_params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'learning_rate': 0.05,
        'random_state': seed,
    }

    classification_cat_params = {
        'depth':7,
        'learning_rate': 0.01,
        'iterations': 1200,
        'l2_leaf_reg': 7,
        'random_seed': seed,
    }
    classification_adaboost_params = {
        'n_estimators': 100,
        'learning_rate': 1.0,
        'random_state': seed,
    }
    nn_params = {
        'input_size': 62,  # 特徴量の数に応じて変更してください
        'hidden_size': [64, 32],  # 隠れ層のユニット数
        'output_size': 1,  # 出力層のユニット数
        'dropout_rate': 0.1,
        'learning_rate': 0.001,
        'batch_size': 64,
        'epochs': 10,
    }

    model_weight_dict = {'lightgbm': 0.30, 'xgboost': 0.10, 'catboost': 0.30, 'adaboost': 0.10, 'neuralnetwork': 0.15}
# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(CFG.seed)


# ====================================================
# Metric
# ====================================================
# f1_score

# ====================================================
# LightGBM Metric
# ====================================================
def lgb_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'f1score', f1_score(y_true, np.where(y_pred >= 0.5, 1, 0), average='macro'), CFG.metric_maximize_flag

# ====================================================
# XGBoost Metric
# ====================================================
def xgb_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'f1score', f1_score(y_true, np.where(y_pred >= 0.5, 1, 0), average='macro')

class SimpleNN(nn.Module):
    def __init__(self, cfg):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(cfg['input_size'], cfg['hidden_size'][0])
        self.dropout = nn.Dropout(cfg['dropout_rate'])
        self.fc2 = nn.Linear(cfg['hidden_size'][0], cfg['hidden_size'][1])
        self.fc3 = nn.Linear(cfg['hidden_size'][1], cfg['output_size'])
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x
    
def f1_score_nn(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.round(K.flatten(y_pred))
    tp = K.sum(y_true * y_pred)
    fp = K.sum((1 - y_true) * y_pred)
    fn = K.sum(y_true * (1 - y_pred))
    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    return f1

def macro_f1_score_nn(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.round(K.flatten(y_pred))
    tp = K.sum(y_true * y_pred)
    fp = K.sum((1 - y_true) * y_pred)
    fn = K.sum(y_true * (1 - y_pred))
    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    
    # Calculate F1 score for each class
    f1_per_class = []
    for c in range(tf.shape(y_true)[-1]):
        true_positives = K.sum(K.cast(y_true[:, c] * K.round(y_pred[:, c]), 'float'), axis=0)
        possible_positives = K.sum(K.cast(y_true[:, c], 'float'), axis=0)
        predicted_positives = K.sum(K.cast(K.round(y_pred[:, c]), 'float'), axis=0)
        precision = true_positives / (predicted_positives + K.epsilon())
        recall = true_positives / (possible_positives + K.epsilon())
        f1_per_class.append(2 * precision * recall / (precision + recall + K.epsilon()))
        
class MacroF1ScoreCallback(Callback):
    def __init__(self, validation_data):
        super(MacroF1ScoreCallback, self).__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        val_targ = self.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict, average='macro')
        print(f'Epoch {epoch+1}: val_macro_f1: {_val_f1}')
    


In [132]:
#LGB用データ作成
#データの読み込み
train_df = pd.read_csv('train.csv', index_col=0)
test_df = pd.read_csv('test.csv', index_col=0)
categorical_features = ['RevLineCr', 'LowDoc', 'UrbanRural', 'State', 'Sector','DisbursementDate','ApprovalDate']

#前処理メソッドの定義
def Preprocessing(input_df: pd.DataFrame()) -> pd.DataFrame():
    #欠損値に対する処理
    def deal_missing(input_df: pd.DataFrame()) -> pd.DataFrame():
        df = input_df.copy()
        for col in ['RevLineCr', 'LowDoc', 'BankState']:
            df[col] = input_df[col].fillna('UNK')
        for col in ['DisbursementDate','ApprovalDate']:
            df[col] = input_df[col].fillna('50-NaN-50')
        
        return df
    #金額に対する前処理
    def clean_money(input_df: pd.DataFrame()) -> pd.DataFrame():
        df = input_df.copy()
        for col in ['DisbursementGross', 'GrAppv', 'SBA_Appv']:
            df[col] = input_df[col].str[1:].str.replace(',', '').str.replace(' ', '').astype(float)
        return df
    df = deal_missing(input_df)
    df = clean_money(df)
    df['NewExist'] = np.where(input_df['NewExist'] == 1, 1, 0)
    #特徴量作成
    def make_features(input_df: pd.DataFrame()) -> pd.DataFrame():
        df = input_df.copy()
        #日付関係の特徴量作成
        df[['DisbursementDay','DisbursementMonth','DisbursementYear']] = df['DisbursementDate'].str.split('-',expand=True)
        df[['ApprovalDay','ApprovalMonth','ApprovalYear']] = df['ApprovalDate'].str.split('-',expand=True)
        df['DisbursementDay'] = df['DisbursementDay'].astype(int)
        df['DisbursementYear'] = df['DisbursementYear'].astype(int)
        df['ApprovalDay'] = df['ApprovalDay'].astype(int)
        df['ApprovalYear'] = df['ApprovalYear'].astype(int)
        Month_dict = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12,'NaN':50}
        df['DisbursementMonth'] = df['DisbursementMonth'].map(Month_dict)
        df['ApprovalMonth'] = df['ApprovalMonth'].map(Month_dict)
        df['DisbursementDate'] = df['DisbursementYear'].astype(str)+df['DisbursementMonth'].astype(str)+df['DisbursementDay'].astype(str)
        df['DisbursementYear'] = df['DisbursementYear'].apply(lambda x:x - 100 if x >50 else x)
        df['ApprovalYear'] = df['ApprovalYear'].apply(lambda x:x - 100 if x >50 else x)
        df['CompanyLong'] = df['DisbursementYear'] - df['ApprovalYear']
        df['ApprovalTerm'] = 15 - df['ApprovalFY']
        df['DisbursementTerm'] = 15 - df['DisbursementYear']

    
        #Bankraptdataの74~80は生成したもので実際の数値ではない。(失業率から換算して生成)
        Bankraptcydata={-26:32700,-25:52200,-24:46200,-23:42300,-22:36300,-21:34200,-20:46200,-19:44000,-18:48500,-17:69800,-16:62500,
                      -15:64500,-14:72000,-13:81500,-12:83000,-11:64500,-10:65000,-9:67000,-8:71000,-7:67000,-6:58000,-5:51000,
                        -4:52500,-3:54000,-2:51000,-1:41000,0:37500,1:35992,2:39845,3:37548,4:36785,5:31952,6:35292,7:21960,8:30741,
                        9:49091,10:61148,11:54212,12:46393,13:37552,14:31671,15:26130,16:24797,17:23591,18:23106,19:22157}
        

        #年ごとのデータを、1-5年後の平均に変換
        datalist = [Bankraptcydata]
        for k in datalist:
            for i in range(len(k)-5):
                k[-27+i] = 0
                for j in range(5):
                    k[-27+i] += k[-26+i+j]
                k[-27+i] = k[-27+i]/5
            k[50] = sum(k.values()) / len(k)
        
            
        
        df['Bankraptcy_By_Year'] = df['DisbursementYear'].map(Bankraptcydata)
        
        
        #State関係の特徴量作成
        df['Cor_State'] = (df['State']==df['BankState']).astype(int)
        StateList = ['AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA',
                      'MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX',
                      'UT','VT','VA','WA','WV','WI','WY']
        
        UnemploymentList = [2.6,3.7,4.0,3.4,4.1,2.8,4.0,4.6,4.2,2.7,3.1,3.7,2.8,4.6,3.1,3.0,2.9,3.9,3.5,3.1,3.0,3.7,4.3,2.9,4.0,
                          2.7,2.6,2.7,5.5,2.9,3.3,3.5,4.1,3.8,2.1,4.1,3.2,4.8,4.3,3.2,3.3,2.2,3.5,3.8,2.4,3.0,3.1
                            ,4.5,4.1,3.0,3.9]

        GDPList = [29603,44807,33655,27781,42376,40805,51911,56496,126421,33417,35265,38850,29843,39568,32724,35814,34770,30364,35181,
                   30282,39596,47351,32846,41353,24477,32590,28201,37075,40210,37375,45052,30943,49038,37053,34694,34040,29470,
                   38339,35153,36543,28894,35596,33742,37793,32774,34197,41617,40361,24929,34890,40303]
        
        GDPperPersonList = [37282,71008,48148,35674,53525,54943,63504,76720,164002,45958,48434,50788,39529,49083,40529,44091,43633,
                       38148,48366,37734,50729,55364,38433,51829,31127,41012,37966,46803,63662,46400,55320,41878,58126,49625,43172,
                       41073,40376,46248,43246,44738,38093,44955,42865,54766,47313,40312,54102,52810,31914,43309,63822]
        
        AveSalaryList = [40.46,50.81,45.40,37.79,56.10,49.79,60.14,49.66,79.85,43.66,46.17,44.09,36.45,51.71,40.97,38.39,40.96,
                        39.54,43.15,39.06,54.28,58.62,45.19,46.99,35.95,42.58,35.81,39.87,44.38,46.38,56.72,40.91,61.04,43.11,41.12,
                        43.45,40.75,43.46,46.10,46.38,39.63,35.00,41.88,48.35,41.11,39.54,52.07,51.04,38.48,41.46,44.03]
        
        Unemploymentdict = dict(zip(StateList,UnemploymentList))
        GDPdict = dict(zip(StateList,GDPList))
        GDPperPersondict = dict(zip(StateList,GDPperPersonList))
        AveSalarydict = dict(zip(StateList,AveSalaryList))
        
        df['Unemployment_By_State'] = df['State'].map(Unemploymentdict)
        df['GDP_By_State'] = df['State'].map(GDPdict)
        df['GDPperPerson_By_State'] = df['State'].map(GDPperPersondict)
        df['AveSalary_By_State'] = df['State'].map(AveSalarydict)
        
        
        #現状グループ分けされない特徴量の作成
        #企業の安定さ、デカさ
        df['BCI'] = df['CompanyLong']*(df['NoEmp'])*(df['NewExist']+1)
        df['BCI'] = df['BCI'].fillna(df['BCI'].mean)
        #しんどさ指数
        df['TI'] = (df['DisbursementGross']/(df['NoEmp']+df['CreateJob']+1))/(df['Term']+1)
        #しんどさ指数2
        df['TI2'] = (df['SBA_Appv']/(df['NoEmp']+df['CreateJob']+1))/(df['Term']+1)
        
        #派生特徴量
        # Employee to Loan Size Ratio
        # 財務関連の派生特徴量
        df['GrAppv_SBA_Appv_ratio'] = df['SBA_Appv'] / (df['GrAppv'] + 1) 
        df['DisbursementGross_GrAppv_ratio'] = df['DisbursementGross'] / (df['GrAppv'] + 1)
        # ビジネスの条件関連
        df['NewExist_NoEmp_interaction'] = df['NewExist'] * df['NoEmp']
        df['UrbanRural_Sector_interaction'] = df['UrbanRural'].astype(str) + '_' + df['Sector'].astype(str)
        # リスク関連の派生特徴量
        df['RevLineCr_LowDoc_risk_indicator'] = (df['RevLineCr'] == 'Y').astype(int) + (df['LowDoc'] == 'Y').astype(int)
        # FranchiseCodeのリスク要因
        df['Franchise_risk_factor'] = df['FranchiseCode'].apply(lambda x: 0 if x in [0, 1] else 1)
        # 経済的特徴の組み合わせ
        df['Emp_to_Loan_Ratio'] = df['NoEmp'] / (df['DisbursementGross'] + 1)
        df['JobImpactScore'] = df['CreateJob'] + df['RetainedJob']
        df['Employment_creation_ratio'] = df['CreateJob'] / (df['NoEmp'] + 1)
        df['Disbursement_per_Term'] = df['DisbursementGross'] / (df['Term']+1)
        # 業種と金融条件の組み合わせ
        df['Sector_RevLineCr'] = df['Sector'].astype(str) + '_' + df['RevLineCr']
        df['Sector_LowDoc'] = df['Sector'].astype(str) + '_' + df['LowDoc']
        


        


        
        return df
    df = make_features(df)
    return df

#前処理の実行
train_df = Preprocessing(train_df)
test_df = Preprocessing(test_df)

#カウントエンコーディング
for col in ['FranchiseCode','UrbanRural', 'State','City', 'Sector', 
            'UrbanRural_Sector_interaction', 'Sector_RevLineCr', 'Sector_LowDoc']:
    count_dict = dict(train_df[col].value_counts())
    train_df[f'{col}_count_encoding'] = train_df[col].map(count_dict).astype(int)
    test_df[f'{col}_count_encoding'] = test_df[col].map(count_dict).fillna(1).astype(int)

#ラベルエンコーディング
for col in categorical_features:
    encoder = LabelEncoder()
    combined = pd.concat([train_df[col], test_df[col]], axis=0)
    encoder.fit(combined)
    train_df[col] = encoder.transform(train_df[col])
    test_df[col] = encoder.transform(test_df[col])
    
#OneHotEncoding
train_df2 = train_df.drop(['MIS_Status'],axis=1)
OneHotList = ['RevLineCr', 'LowDoc']
ohe = ce.OneHotEncoder(cols=OneHotList,use_cat_names=True)
train_df2 = ohe.fit_transform(train_df2)
test_df = ohe.transform(test_df)
train_df = pd.concat([train_df2,train_df['MIS_Status']],axis=1)

#featuresの作成
categorical_features =['UrbanRural', 'State', 'Sector','RevLineCr_0.0','RevLineCr_1.0','RevLineCr_2.0','RevLineCr_3.0','RevLineCr_4.0',
                       'LowDoc_0.0','LowDoc_1.0','LowDoc_2.0','LowDoc_3.0','LowDoc_4.0','LowDoc_5.0','LowDoc_6.0','FranchiseCode']

RemoveList=['MIS_Status','City','BankState','UrbanRural_Sector_interaction',
            'Sector_RevLineCr', 'Sector_LowDoc']
features = train_df.columns.tolist()
for i in RemoveList:
    print(i)
    features.remove(i)
    
print(train_df)
#専用変数として保存
train_df_lgb = train_df
test_df_lgb = test_df
categorical_features_lgb = categorical_features
features_lgb = features

MIS_Status
City
BankState
UrbanRural_Sector_interaction
Sector_RevLineCr
Sector_LowDoc
       Term  NoEmp  NewExist  CreateJob  RetainedJob  FranchiseCode  \
0       163     21         1          0            0              1   
1        84      6         1          4            0              0   
2       242     45         1          4           90              0   
3       237      4         1          0            0              0   
4       184      0         1          0            0              0   
...     ...    ...       ...        ...          ...            ...   
42302   283     14         1          0            0              1   
42303    53      2         1          0            0              0   
42304    59      6         0          0            0              1   
42305   295     18         1          0            8              0   
42306    84      4         1          0            8              0   

       RevLineCr_1.0  RevLineCr_0.0  RevLineCr_4.0  RevLineC

In [133]:
#CTB用データ作成
#データの読み込み
train_df = pd.read_csv('train.csv', index_col=0)
test_df = pd.read_csv('test.csv', index_col=0)
default_numerical_features = ['Term', 'NoEmp', 'CreateJob', 'RetainedJob', 'DisbursementGross', 'GrAppv', 'SBA_Appv', 'ApprovalFY']
default_categorical_features = ['RevLineCr', 'LowDoc', 'UrbanRural', 'State', 'Sector','FranchiseCode','City','ApprovalDate',
                                'DisbursementDate','BankState','NewExist']
new_categorical_features = []
#前処理メソッドの定義
def Preprocessing(input_df: pd.DataFrame()) -> pd.DataFrame():
    #欠損値に対する処理
    def deal_missing(input_df: pd.DataFrame()) -> pd.DataFrame():
        df = input_df.copy()
        for col in ['RevLineCr', 'LowDoc', 'BankState']:
            df[col] = input_df[col].fillna('UNK')
        for col in ['DisbursementDate','ApprovalDate']:
            df[col] = input_df[col].fillna('50-NaN-50')
        
        return df
    #金額に対する前処理
    def clean_money(input_df: pd.DataFrame()) -> pd.DataFrame():
        df = input_df.copy()
        for col in ['DisbursementGross', 'GrAppv', 'SBA_Appv']:
            df[col] = input_df[col].str[1:].str.replace(',', '').str.replace(' ', '').astype(float)
        return df
    df = deal_missing(input_df)
    df = clean_money(df)
    df['NewExist'] = np.where(input_df['NewExist'] == 1, 1, 0)
    #特徴量作成
    def make_features(input_df: pd.DataFrame()) -> pd.DataFrame():
        df = input_df.copy()
        #日付関係の特徴量作成
        df[['DisbursementDay','DisbursementMonth','DisbursementYear']] = df['DisbursementDate'].str.split('-',expand=True)
        df[['ApprovalDay','ApprovalMonth','ApprovalYear']] = df['ApprovalDate'].str.split('-',expand=True)
        df['DisbursementDay'] = df['DisbursementDay'].astype(int)
        df['DisbursementYear'] = df['DisbursementYear'].astype(int)
        df['ApprovalDay'] = df['ApprovalDay'].astype(int)
        df['ApprovalYear'] = df['ApprovalYear'].astype(int)
        Month_dict = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12,'NaN':50}
        df['DisbursementMonth'] = df['DisbursementMonth'].map(Month_dict)
        df['ApprovalMonth'] = df['ApprovalMonth'].map(Month_dict)
        df['DisbursementYear'] = df['DisbursementYear'].apply(lambda x:x - 100 if x >50 else x)
        df['ApprovalYear'] = df['ApprovalYear'].apply(lambda x:x - 100 if x >50 else x)
        
        #Bankraptcydataの74~80は生成したもので実際の数値ではない。(失業率から換算して生成)
        Bankraptcydata={-26:32700,-25:52200,-24:46200,-23:42300,-22:36300,-21:34200,-20:46200,-19:44000,-18:48500,-17:69800,-16:62500,
                      -15:64500,-14:72000,-13:81500,-12:83000,-11:64500,-10:65000,-9:67000,-8:71000,-7:67000,-6:58000,-5:51000,
                        -4:52500,-3:54000,-2:51000,-1:41000,0:37500,1:35992,2:39845,3:37548,4:36785,5:31952,6:35292,7:21960,8:30741,
                        9:49091,10:61148,11:54212,12:46393,13:37552,14:31671,15:26130,16:24797,17:23591,18:23106,19:22157}

        #年ごとのデータを、1-5年後の平均に変換
        datalist = [Bankraptcydata]
        for k in datalist:
            for i in range(len(k)-5):
                k[-27+i] = 0
                for j in range(5):
                    k[-27+i] += k[-26+i+j]
                k[-27+i] = k[-27+i]/5
            k[50] = k[-26]*2
        
        df['Bankraptcy_By_Year'] = df['DisbursementYear'].map(Bankraptcydata)

        '''
        #組み合わせ特徴量
        CList = ['Term','NoEmp','CreateJob','RetainedJob','FranchiseCode','RevLineCr','LowDoc','DisbursementDate','ApprovalDate','City',
                'State','BankState','DisbursementGross','GrAppv','SBA_Appv','UrbanRural','NewExist']
        for i in range(len(CList)-2):
            for j in range(i+1,len(CList)):
                a,b = CList[i],CList[j]
                df[f'{a}_{b}'] = df[a].astype(str) + '_' + df[b].astype(str)
        '''
        #組み合わせ特徴量
        df['State_Sector'] = df['State'].astype(str) + '_' + df['Sector'].astype(str)
         # 地理的特徴の組み合わせ
        df['City_State'] = df['City'] + '_' + df['State']
        # 時間的特徴の組み合わせ
        df['ApprovalFY_Term'] = df['ApprovalFY'].astype(str) + '_' + df['Term'].astype(str)
        
        df['FranchiseCode_ApprovalDate'] = df['FranchiseCode'].astype(str) + '_' + df['ApprovalDate'].astype(str)
        
        df['Term_NoEmp'] = df['Term'].astype(str) + '_' + df['NoEmp'].astype(str)
        
        df['City_BankState'] = df['City'].astype(str) + '_' + df['BankState'].astype(str)
        
        df['NoEmp_SBA_Appv'] = df['NoEmp'].astype(str) + '_' + df['SBA_Appv'].astype(str)
        
        df['DisbursementDate_UrbanRural'] = df['DisbursementDate'].astype(str) + '_' + df['UrbanRural'].astype(str)
        
    
    
        
        
            
                


        
        return df
    df = make_features(df)
    return df

#前処理の実行
train_df = Preprocessing(train_df)
test_df = Preprocessing(test_df)

'''
#カウントエンコーディング
for col in categorical_features:
    count_dict = dict(train_df[col].value_counts())
    train_df[f'{col}_count_encoding'] = train_df[col].map(count_dict).astype(int)
    test_df[f'{col}_count_encoding'] = test_df[col].map(count_dict).fillna(1).astype(int)
'''

#ラベルエンコーディング
new_categorical_features = ['State_Sector','City_State','ApprovalFY_Term','FranchiseCode_ApprovalDate','Term_NoEmp',
                           'City_BankState','NoEmp_SBA_Appv','DisbursementDate_UrbanRural']
categorical_features = default_categorical_features + new_categorical_features
for col in categorical_features:
    encoder = LabelEncoder()
    combined = pd.concat([train_df[col], test_df[col]], axis=0)
    encoder.fit(combined)
    train_df[col] = encoder.transform(train_df[col])
    test_df[col] = encoder.transform(test_df[col])
    

#featuresの作成
numerical_features = default_numerical_features + ['Bankraptcy_By_Year']
features = numerical_features + categorical_features 

RemoveList=['MIS_Status','ApprovalDay','ApprovalMonth','ApprovalFY','ApprovalYear','DisbursementDay','DisbursementMonth',
           'DisbursementYear']
features = train_df.columns.tolist()
for i in RemoveList:
    print(i)
    features.remove(i)
    
#専用変数の作成
train_df_ctb = train_df
test_df_ctb = test_df
categorical_features_ctb = categorical_features
features_ctb = features

MIS_Status
ApprovalDay
ApprovalMonth
ApprovalFY
ApprovalYear
DisbursementDay
DisbursementMonth
DisbursementYear


In [134]:
#NN用データ作成
#データの読み込み
train_df = pd.read_csv('train.csv', index_col=0)
test_df = pd.read_csv('test.csv', index_col=0)
categorical_features = ['RevLineCr', 'LowDoc', 'UrbanRural', 'State', 'Sector']
#前処理メソッドの定義
#前処理メソッドの定義
def Preprocessing(input_df: pd.DataFrame()) -> pd.DataFrame():
    #欠損値に対する処理
    def deal_missing(input_df: pd.DataFrame()) -> pd.DataFrame():
        df = input_df.copy()
        for col in ['RevLineCr', 'LowDoc', 'BankState']:
            df[col] = input_df[col].fillna('UNK')
        for col in ['DisbursementDate','ApprovalDate']:
            df[col] = input_df[col].fillna('50-NaN-50')
        
        return df
    #金額に対する前処理
    def clean_money(input_df: pd.DataFrame()) -> pd.DataFrame():
        df = input_df.copy()
        for col in ['DisbursementGross', 'GrAppv', 'SBA_Appv']:
            df[col] = input_df[col].str[1:].str.replace(',', '').str.replace(' ', '').astype(float)
        return df
    df = deal_missing(input_df)
    df = clean_money(df)
    df['NewExist'] = np.where(input_df['NewExist'] == 1, 1, 0)
    #特徴量作成
    def make_features(input_df: pd.DataFrame()) -> pd.DataFrame():
        df = input_df.copy()
        #日付関係の特徴量作成
        df[['DisbursementDay','DisbursementMonth','DisbursementYear']] = df['DisbursementDate'].str.split('-',expand=True)
        df[['ApprovalDay','ApprovalMonth','ApprovalYear']] = df['ApprovalDate'].str.split('-',expand=True)
        df['DisbursementDay'] = df['DisbursementDay'].astype(int)
        df['DisbursementYear'] = df['DisbursementYear'].astype(int)
        df['ApprovalDay'] = df['ApprovalDay'].astype(int)
        df['ApprovalYear'] = df['ApprovalYear'].astype(int)
        Month_dict = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12,'NaN':50}
        df['DisbursementMonth'] = df['DisbursementMonth'].map(Month_dict)
        df['ApprovalMonth'] = df['ApprovalMonth'].map(Month_dict)
        df['DisbursementDate'] = df['DisbursementYear'].astype(str)+df['DisbursementMonth'].astype(str)+df['DisbursementDay'].astype(str)
        df['DisbursementYear'] = df['DisbursementYear'].apply(lambda x:x - 100 if x >50 else x)
        df['ApprovalYear'] = df['ApprovalYear'].apply(lambda x:x - 100 if x >50 else x)
        df['CompanyLong'] = df['DisbursementYear'] - df['ApprovalYear']
        df['ApprovalTerm'] = 15 - df['ApprovalFY']
        df['DisbursementTerm'] = 15 - df['DisbursementYear']


        #経済成長率
        EconomyGrowthdata={-26:-0.6,-25:-0.4,-24:5.6,-23:4.6,-22:5.5,-21:3.2,-20:-0.26,-19:2.54,-18:-1.8,-17:4.58,-16:7.24,-15:4.17,
                           -14:3.46,-13:3.46,-12:4.18,-11:3.67,-10:1.89,-9:-0.11,-8:3.52,-7:2.75,-6:4.03,-5:2.68,-4:3.77,-3:4.45,
                           -2:4.18,-1:4.8,0:4.08,1:0.95,2:1.7,3:2.8,4:3.85,5:3.48,6:2.78,7: 2.01,8:0.12,9:-2.6,10:2.71,11:1.55,12:2.28,
                           13:1.84,14:2.29,15:2.71,16:1.67,17:2.24,18:2.95,19:2.30}
        #Bankraptdataの74~80は生成したもので実際の数値ではない。(失業率から換算して生成)
        Bankraptcydata={-26:32700,-25:52200,-24:46200,-23:42300,-22:36300,-21:34200,-20:46200,-19:44000,-18:48500,-17:69800,-16:62500,
                      -15:64500,-14:72000,-13:81500,-12:83000,-11:64500,-10:65000,-9:67000,-8:71000,-7:67000,-6:58000,-5:51000,
                        -4:52500,-3:54000,-2:51000,-1:41000,0:37500,1:35992,2:39845,3:37548,4:36785,5:31952,6:35292,7:21960,8:30741,
                        9:49091,10:61148,11:54212,12:46393,13:37552,14:31671,15:26130,16:24797,17:23591,18:23106,19:22157}
        #失業率
        Unemploymentratedata={-26:5.45,-25:8.7,-24:7.7,-23:7.05,-22:6.05,-21:5.7,-20:7.7,-19:7.35,-18:9.7,-17:9.75,-16:7.35,-15:7.4,
                              -14:7.1,-13:6.15,-12:5.4,-11:5.25,-10:5.35,-9:6.85,-8:7.75,-7:6.95,-6:6.1,-5:5.65,-4:5.4,-3:4.95,-2:4.5,
                              -1:4.3,0:4.0,1:4.55,2:5.8,3:6.25,4:5.55,5:5.0,6:4.65,7:4.65,8:5.7,9:9.5,10:9.4,11:9.05,12:8.2,13:7.4,
                              14:6.15,15:5.25,16:4.9,17:4.3,18:3.9,19:3.6}
        #金利
        Interestratedata={-26:10,-25:7,-24:5,-23:7,-22:8.5,-21:12,-20:15,-19:15,-18:14,-17:9,-16:10,-15:8.5,
                              -14:7,-13:6.5,-12:8,-11:9,-10:7,-9:5.5,-8:3.5,-7:3,-6:4,-5:6,-4:5.5,-3:5.7,-2:5.3,
                              -1:5,0:6.1,1:4.3,2:1.8,3:1.1,4:1.8,5:3.5,6:5.2,7:5,8:2,9:0.25,10:0.25,11:0.25,12:0.25,13:0.25,
                              14:0.25,15:0.25,16:0.5,17:1,18:2,19:2.2}
        #インフレ率
        Inflationratedata={-26:11,-25:9,-24:5,-23:7,-22:8,-21:12,-20:13.5,-19:10.38,-18:6.16,-17:3.16,-16:4.37,-15:3.16,
                           -14:1.94,-13:3.58,-12:4.1,-11:4.79,-10:5.42,-9:4.22,-8:3.04,-7:2.97,-6:2.6,-5:2.81,-4:2.94,-3:2.34,
                           -2:1.55,-1:2.19,0:3.37,1:2.82,2:1.6,3:2.3,4:2.67,5:3.37,6:3.22,7:2.87,8:3.82,9:-0.32,10:1.64,11:3.14,12:2.07,
                           13:1.47,14:1.62,15:0.12,16:1.27,17:2.13,18:2.44,19:1.81}
        #年ごとのデータを、1-5年後の平均に変換
        datalist = [EconomyGrowthdata,Bankraptcydata,Unemploymentratedata,Interestratedata,Inflationratedata]
        for k in datalist:
            for i in range(len(k)-5):
                k[-27+i] = 0
                for j in range(5):
                    k[-27+i] += k[-26+i+j]
                k[-27+i] = k[-27+i]/5
            k[50] = sum(k.values()) / len(k)
        
        df['EconomyGrowth_By_Year'] = df['DisbursementYear'].map(EconomyGrowthdata)
        df['Bankraptcy_By_Year'] = df['DisbursementYear'].map(Bankraptcydata)
        df['Unemploymentrate_By_Year'] = df['DisbursementYear'].map(Unemploymentratedata)
        df['Interestrate_By_Year'] = df['DisbursementYear'].map(Interestratedata)
        df['Inflationrate_By_Year'] = df['DisbursementYear'].map(Inflationratedata)
        
        #State関係の特徴量作成
        StateList = ['AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA',
                      'MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX',
                      'UT','VT','VA','WA','WV','WI','WY']
        
        UnemploymentList = [2.6,3.7,4.0,3.4,4.1,2.8,4.0,4.6,4.2,2.7,3.1,3.7,2.8,4.6,3.1,3.0,2.9,3.9,3.5,3.1,3.0,3.7,4.3,2.9,4.0,
                          2.7,2.6,2.7,5.5,2.9,3.3,3.5,4.1,3.8,2.1,4.1,3.2,4.8,4.3,3.2,3.3,2.2,3.5,3.8,2.4,3.0,3.1
                            ,4.5,4.1,3.0,3.9]
        
        GDPList = [29603,44807,33655,27781,42376,40805,51911,56496,126421,33417,35265,38850,29843,39568,32724,35814,34770,30364,35181,
                   30282,39596,47351,32846,41353,24477,32590,28201,37075,40210,37375,45052,30943,49038,37053,34694,34040,29470,
                   38339,35153,36543,28894,35596,33742,37793,32774,34197,41617,40361,24929,34890,40303]
        
        GDPperPersonList = [37282,71008,48148,35674,53525,54943,63504,76720,164002,45958,48434,50788,39529,49083,40529,44091,43633,
                       38148,48366,37734,50729,55364,38433,51829,31127,41012,37966,46803,63662,46400,55320,41878,58126,49625,43172,
                       41073,40376,46248,43246,44738,38093,44955,42865,54766,47313,40312,54102,52810,31914,43309,63822]
        
        AveSalaryList = [40.46,50.81,45.40,37.79,56.10,49.79,60.14,49.66,79.85,43.66,46.17,44.09,36.45,51.71,40.97,38.39,40.96,
                        39.54,43.15,39.06,54.28,58.62,45.19,46.99,35.95,42.58,35.81,39.87,44.38,46.38,56.72,40.91,61.04,43.11,41.12,
                        43.45,40.75,43.46,46.10,46.38,39.63,35.00,41.88,48.35,41.11,39.54,52.07,51.04,38.48,41.46,44.03]
        
        Unemploymentdict = dict(zip(StateList,UnemploymentList))
        GDPdict = dict(zip(StateList,GDPList))
        GDPperPersondict = dict(zip(StateList,GDPperPersonList))
        AveSalarydict = dict(zip(StateList,AveSalaryList))
        
        df['Unemployment_By_State'] = df['State'].map(Unemploymentdict)
        df['GDP_By_State'] = df['State'].map(GDPdict)
        df['GDPperPerson_By_State'] = df['State'].map(GDPperPersondict)
        df['AveSalary_By_State'] = df['State'].map(AveSalarydict)
        
        #現状グループ分けされない特徴量の作成
        #企業の安定さ、デカさ
        df['BCI'] = df['CompanyLong']*(df['NoEmp'])*(df['NewExist']+1)
        df['BCI'] = df['BCI'].fillna(df['BCI'].mean)
        #一か月あたりの返済必要量
        df['DisbursementGrossPerMonth'] = df['DisbursementGross']/(df['Term']+1)
        #SBA承認より減らした額
        df['SBA_Appv-DisbursementGross'] = df['SBA_Appv']-df['DisbursementGross']
        #本来の従業員一人当たりの返済必要量
        df['DisbursementGrossPerNoEmp'] = df['DisbursementGross']/(df['NoEmp']+1)
        #雇用創出後の従業員一人当たりの返済必要量
        df['DisbursementGrossPerEmp'] = df['DisbursementGross']/(df['NoEmp']+df['CreateJob']+1)
        #しんどさ指数
        df['TI'] = (df['DisbursementGross']/(df['NoEmp']+df['CreateJob']+1))/(df['Term']+1)
        #しんどさ指数2
        df['TI2'] = (df['SBA_Appv']/(df['NoEmp']+df['CreateJob']+1))/(df['Term']+1)


        
        return df
    df = make_features(df)
    return df

#前処理の実行
train_df = Preprocessing(train_df)
test_df = Preprocessing(test_df)

#ラベルエンコーディング
for col in categorical_features :
    le = LabelEncoder()
    le.fit(train_df[col])
    train_df[col] = le.transform(train_df[col])
    test_df[col] = le.transform(test_df[col])
    
categorical_features_unlabelable = ['City','ApprovalDate','BankState','DisbursementDate']
'''
for col in categorical_features_unlabelable:
    le = LabelEncoder()   
    le.fit(train_df[col])
    train_df[col] = le.transform(train_df[col])
    test_df[col] = test_df[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else len(le.classes_))
'''
for col in categorical_features_unlabelable:
    encoder = LabelEncoder()
    combined = pd.concat([train_df[col], test_df[col]], axis=0)
    encoder.fit(combined)
    train_df[col] = encoder.transform(train_df[col])
    test_df[col] = encoder.transform(test_df[col])
    
#OneHotEncoding
train_df2 = train_df.drop(['MIS_Status'],axis=1)
OneHotList = ['RevLineCr', 'LowDoc']
ohe = ce.OneHotEncoder(cols=OneHotList,use_cat_names=True)
train_df2 = ohe.fit_transform(train_df2)
test_df = ohe.transform(test_df)
train_df = pd.concat([train_df2,train_df['MIS_Status']],axis=1)

#featuresの作成
categorical_features = ['State', 'Sector','RevLineCr_0.0','RevLineCr_1.0','RevLineCr_2.0','RevLineCr_3.0','RevLineCr_4.0',
                       'LowDoc_0.0','LowDoc_1.0','LowDoc_2.0','LowDoc_3.0','LowDoc_4.0','LowDoc_5.0','LowDoc_6.0','UrbanRural']


RemoveList=['MIS_Status','City','ApprovalDate','BankState','DisbursementDate','ApprovalDay','ApprovalMonth','ApprovalFY','ApprovalYear',
           'DisbursementDay','DisbursementMonth']
features = train_df.columns.tolist()
for i in RemoveList:
    print(i)
    features.remove(i)
OneHotedList = ['RevLineCr_0.0','RevLineCr_1.0','RevLineCr_2.0','RevLineCr_3.0','RevLineCr_4.0','LowDoc_0.0','LowDoc_1.0','LowDoc_2.0','LowDoc_3.0','LowDoc_4.0','LowDoc_5.0','LowDoc_6.0']

scalelist = features
for i in OneHotedList:
    scalelist.remove(i)
    
stdscl = StandardScaler()
train_df[scalelist] = stdscl.fit_transform(train_df[scalelist])
test_df[scalelist] = stdscl.fit_transform(test_df[scalelist])

RemoveList=['MIS_Status','City','ApprovalDate','BankState','DisbursementDate','ApprovalDay','ApprovalMonth','ApprovalFY','ApprovalYear',
           'DisbursementDay','DisbursementMonth']
features = train_df.columns.tolist()
for i in RemoveList:
    print(i)
    features.remove(i)

    
print(train_df)
train_df.info()
print(features)

train_df_nn = train_df
test_df_nn = test_df
features_nn = features


MIS_Status
City
ApprovalDate
BankState
DisbursementDate
ApprovalDay
ApprovalMonth
ApprovalFY
ApprovalYear
DisbursementDay
DisbursementMonth
MIS_Status
City
ApprovalDate
BankState
DisbursementDate
ApprovalDay
ApprovalMonth
ApprovalFY
ApprovalYear
DisbursementDay
DisbursementMonth
           Term     NoEmp  NewExist  CreateJob  RetainedJob  FranchiseCode  \
0      0.643243  0.645918  0.516224  -0.428693    -0.428684      -0.185372   
1     -0.290907 -0.211822  0.516224   0.356557    -0.428684      -0.185467   
2      1.577393  2.018302  0.516224   0.356557    10.663868      -0.185467   
3      1.518270 -0.326187  0.516224  -0.428693    -0.428684      -0.185467   
4      0.891561 -0.554918  0.516224  -0.428693    -0.428684      -0.185467   
...         ...       ...       ...        ...          ...            ...   
42302  2.062205  0.245639  0.516224  -0.428693    -0.428684      -0.185372   
42303 -0.657472 -0.440553  0.516224  -0.428693    -0.428684      -0.185467   
42304 -0.586524 -0

In [135]:
# AdaBoost training
def adaboost_training(x_train: pd.DataFrame, y_train: pd.DataFrame, x_valid: pd.DataFrame, y_valid: pd.DataFrame, features: list, categorical_features: list):
    model = AdaBoostClassifier(**CFG.classification_adaboost_params)
    model.fit(x_train, y_train)
    valid_pred = model.predict_proba(x_valid)[:, 1]
    return model, valid_pred


def build_model(input_shape):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(input_shape,)))
    model.add(Dropout(0.1))  # Dropout層を追加
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.1))  
    model.add(Dense(1, activation='sigmoid')) 

    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# モデルの学習と評価
def nn_training(x_train: pd.DataFrame, y_train: pd.DataFrame, x_valid: pd.DataFrame, y_valid: pd.DataFrame, features: list, categorical_features: list):
    model = build_model(x_train.shape[1])
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(x_train, y_train, epochs=100, batch_size=32, verbose=1,
          validation_data=(x_valid, y_valid),
          callbacks=[MacroF1ScoreCallback(validation_data=(x_valid, y_valid)), early_stopping])
    valid_pred = model.predict(x_valid)
    valid_pred = valid_pred.flatten()  # 1 次元の配列に変換する
    return  model, valid_pred

#lightgbmでの学習メソッドの定義
def lightgbm_training(x_train: pd.DataFrame, y_train: pd.DataFrame, x_valid: pd.DataFrame, y_valid: pd.DataFrame, features: list, categorical_features: list):
    lgb_train = lgb.Dataset(x_train, y_train, categorical_feature=categorical_features)
    lgb_valid = lgb.Dataset(x_valid, y_valid, categorical_feature=categorical_features)
    model = lgb.train(
                params = CFG.classification_lgb_params,
                train_set = lgb_train,
                num_boost_round = CFG.num_boost_round,
                valid_sets = [lgb_train, lgb_valid],
                feval = lgb_metric,
                callbacks=[lgb.early_stopping(stopping_rounds=CFG.early_stopping_round,
                                              verbose=CFG.verbose)]
            )
    # Predict validation
    valid_pred = model.predict(x_valid)
    return model, valid_pred

#xgboostでの学習メソッドの定義
def xgboost_training(x_train: pd.DataFrame, y_train: pd.DataFrame, x_valid: pd.DataFrame, y_valid: pd.DataFrame, features: list, categorical_features: list):
    xgb_train = xgb.DMatrix(data=x_train, label=y_train)
    xgb_valid = xgb.DMatrix(data=x_valid, label=y_valid)
    model = xgb.train(
                CFG.classification_xgb_params,
                dtrain = xgb_train,
                num_boost_round = CFG.num_boost_round,
                evals = [(xgb_train, 'train'), (xgb_valid, 'eval')],
                early_stopping_rounds = CFG.early_stopping_round,
                verbose_eval = CFG.verbose,
                feval = xgb_metric,
                maximize = CFG.metric_maximize_flag,
            )
    # Predict validation
    valid_pred = model.predict(xgb.DMatrix(x_valid))
    return model, valid_pred

#catboostでの学習メソッドの定義
def catboost_training(x_train: pd.DataFrame, y_train: pd.DataFrame, x_valid: pd.DataFrame, y_valid: pd.DataFrame, features: list, categorical_features: list):
    cat_train = Pool(data=x_train, label=y_train, cat_features=categorical_features)
    cat_valid = Pool(data=x_valid, label=y_valid, cat_features=categorical_features)
    model = CatBoostClassifier(**CFG.classification_cat_params)
    model.fit(cat_train,
              eval_set = [cat_valid],
              early_stopping_rounds = CFG.early_stopping_round,
              verbose = CFG.verbose,
              use_best_model = True)
    # Predict validation
    valid_pred = model.predict_proba(x_valid)[:, 1]
    return model, valid_pred
#任意のモデルでのクロスバリデーション学習メソッドの定義
def gradient_boosting_model_cv_training(method: str, train_df: pd.DataFrame, features: list, categorical_features: list):
    # Create a numpy array to store out of folds predictions
    oof_predictions = np.zeros(len(train_df))
    oof_fold = np.zeros(len(train_df))
    kfold = KFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.seed)
    for fold, (train_index, valid_index) in enumerate(kfold.split(train_df)):
        print('-'*50)
        print(f'{method} training fold {fold+1}')

        x_train = train_df[features].iloc[train_index]
        y_train = train_df[CFG.target_col].iloc[train_index]
        x_valid = train_df[features].iloc[valid_index]
        y_valid = train_df[CFG.target_col].iloc[valid_index]
        
        model = None  # モデル変数を初期化する
        valid_pred = None
        
        if method == 'adaboost':
            model, valid_pred = adaboost_training(x_train, y_train, x_valid, y_valid, features, categorical_features)
            pickle.dump(model, open(f'{method}_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.pkl', 'wb'))
        if method == 'neuralnetwork':
            model, valid_pred = nn_training(x_train, y_train, x_valid, y_valid, features, categorical_features)
            model.save(f'{method}_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.h5')
        if method == 'lightgbm':
            model, valid_pred = lightgbm_training(x_train, y_train, x_valid, y_valid, features, categorical_features)
            pickle.dump(model, open(f'{method}_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.pkl', 'wb'))
        if method == 'xgboost':
            model, valid_pred = xgboost_training(x_train, y_train, x_valid, y_valid, features, categorical_features)
            pickle.dump(model, open(f'{method}_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.pkl', 'wb'))
        if method == 'catboost':
            model, valid_pred = catboost_training(x_train, y_train, x_valid, y_valid, features, categorical_features)
            pickle.dump(model, open(f'{method}_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.pkl', 'wb'))
    
        # Add to out of folds array
        oof_predictions[valid_index] = valid_pred
        oof_fold[valid_index] = fold + 1
        del x_train, x_valid, y_train, y_valid, model, valid_pred
        gc.collect()

    # Compute out of folds metric
    score = f1_score(train_df[CFG.target_col], oof_predictions >= 0.5, average='macro')
    print(f'{method} our out of folds CV f1score is {score}')
    # Create a dataframe to store out of folds predictions
    oof_df = pd.DataFrame({CFG.target_col: train_df[CFG.target_col], f'{method}_prediction': oof_predictions, 'fold': oof_fold})
    oof_df.to_csv(f'oof_{method}_seed{CFG.seed}_ver{CFG.VER}.csv', index = False)
#学習メソッドの定義
def Learning():
    gradient_boosting_model_cv_training('neuralnetwork', train_df_nn, features_nn, categorical_features_lgb)
    gradient_boosting_model_cv_training('adaboost', train_df_ctb, features_ctb, categorical_features_ctb)
    gradient_boosting_model_cv_training('lightgbm', train_df_ctb, features_ctb, categorical_features_ctb)
    gradient_boosting_model_cv_training('xgboost',train_df_ctb, features_ctb, categorical_features_ctb)
    gradient_boosting_model_cv_training('catboost',train_df_ctb, features_ctb, categorical_features_ctb)

In [136]:
#lgb入力をctb入力に変えた

In [137]:
Learning()

--------------------------------------------------
neuralnetwork training fold 1
Epoch 1/100
189/189 [==============================] - 1s 3ms/step loss: 0.3150 -
Epoch 1: val_macro_f1: 0.593023186847424
1134/1134 [==============================] - 8s 6ms/step - loss: 0.3151 - accuracy: 0.8922 - val_loss: 0.2873 - val_accuracy: 0.9042
Epoch 2/100
189/189 [==============================] - 1s 2ms/step loss: 0.2938 -
Epoch 2: val_macro_f1: 0.6110033575448729
1134/1134 [==============================] - 6s 5ms/step - loss: 0.2943 - accuracy: 0.9012 - val_loss: 0.2851 - val_accuracy: 0.9062
Epoch 3/100
189/189 [==============================] - 0s 2ms/step loss: 0.2914 - a
Epoch 3: val_macro_f1: 0.6144611652666098
1134/1134 [==============================] - 6s 5ms/step - loss: 0.2915 - accuracy: 0.9024 - val_loss: 0.2820 - val_accuracy: 0.9060
Epoch 4/100
189/189 [==============================] - 0s 2ms/step loss: 0.2884 - a
Epoch 4: val_macro_f1: 0.6107340428884842
1134/1134 [==========

189/189 [==============================] - 1s 2ms/step loss: 0.3111 - a
Epoch 1: val_macro_f1: 0.6096453490790549
1134/1134 [==============================] - 8s 6ms/step - loss: 0.3111 - accuracy: 0.8939 - val_loss: 0.2933 - val_accuracy: 0.9024
Epoch 2/100
189/189 [==============================] - 0s 2ms/step loss: 0.2924 - acc
Epoch 2: val_macro_f1: 0.6325124713861369
1134/1134 [==============================] - 6s 5ms/step - loss: 0.2927 - accuracy: 0.9017 - val_loss: 0.2907 - val_accuracy: 0.9025
Epoch 3/100
189/189 [==============================] - 1s 2ms/step loss: 0.2902 - a
Epoch 3: val_macro_f1: 0.6253378198471656
1134/1134 [==============================] - 6s 5ms/step - loss: 0.2904 - accuracy: 0.9030 - val_loss: 0.2906 - val_accuracy: 0.9014
Epoch 4/100
189/189 [==============================] - 0s 2ms/step loss: 0.2875 - a
Epoch 4: val_macro_f1: 0.6267390830165919
1134/1134 [==============================] - 6s 5ms/step - loss: 0.2880 - accuracy: 0.9042 - val_loss: 0.28

189/189 [==============================] - 1s 2ms/step loss: 0.3104 - a
Epoch 1: val_macro_f1: 0.609083354483935
1134/1134 [==============================] - 8s 6ms/step - loss: 0.3105 - accuracy: 0.8956 - val_loss: 0.2899 - val_accuracy: 0.9040
Epoch 2/100
189/189 [==============================] - 1s 3ms/step loss: 0.2934 - a
Epoch 2: val_macro_f1: 0.6037177744583229
1134/1134 [==============================] - 6s 5ms/step - loss: 0.2937 - accuracy: 0.9018 - val_loss: 0.2870 - val_accuracy: 0.9042
Epoch 3/100
189/189 [==============================] - 1s 3ms/step loss: 0.2902 
Epoch 3: val_macro_f1: 0.6223671907372494
1134/1134 [==============================] - 6s 5ms/step - loss: 0.2899 - accuracy: 0.9027 - val_loss: 0.2876 - val_accuracy: 0.9047
Epoch 4/100
189/189 [==============================] - 1s 3ms/step loss: 0.2878 - a
Epoch 4: val_macro_f1: 0.6055975218799361
1134/1134 [==============================] - 6s 5ms/step - loss: 0.2878 - accuracy: 0.9037 - val_loss: 0.2860 - v

189/189 [==============================] - 1s 3ms/step loss: 0.2743 -
Epoch 15: val_macro_f1: 0.642113264740894
1134/1134 [==============================] - 6s 5ms/step - loss: 0.2743 - accuracy: 0.9068 - val_loss: 0.2993 - val_accuracy: 0.8992
Epoch 16/100
189/189 [==============================] - 1s 3ms/step loss: 0.2740 -
Epoch 16: val_macro_f1: 0.631896917188629
1134/1134 [==============================] - 6s 6ms/step - loss: 0.2739 - accuracy: 0.9069 - val_loss: 0.2975 - val_accuracy: 0.8987
Epoch 17/100
189/189 [==============================] - 1s 3ms/step loss: 0.2731 - a
Epoch 17: val_macro_f1: 0.6353395178953499
1134/1134 [==============================] - 6s 5ms/step - loss: 0.2731 - accuracy: 0.9070 - val_loss: 0.2994 - val_accuracy: 0.8991
Epoch 18/100
189/189 [==============================] - 1s 3ms/step loss: 0.2735 -
Epoch 18: val_macro_f1: 0.6363070821289023
189/189 [==============================] - 1s 3ms/step
--------------------------------------------------
neur

--------------------------------------------------
lightgbm training fold 3
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 32375, number of negative: 3888
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013070 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 21060
[LightGBM] [Info] Number of data points in the train set: 36263, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.892783 -> initscore=2.119492
[LightGBM] [Info] Start training from score 2.119492
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[18]	training's auc: 0.888175	training's f1score: 0.471677	valid_1's auc: 0.763122	valid_1's f1score: 0.

[200]	train-logloss:0.23038	train-f1score:0.70859	eval-logloss:0.27888	eval-f1score:0.66934
[225]	train-logloss:0.22606	train-f1score:0.71308	eval-logloss:0.27908	eval-f1score:0.66947
[250]	train-logloss:0.22159	train-f1score:0.72172	eval-logloss:0.27941	eval-f1score:0.67120
[275]	train-logloss:0.21903	train-f1score:0.72662	eval-logloss:0.27957	eval-f1score:0.67095
[300]	train-logloss:0.21661	train-f1score:0.73045	eval-logloss:0.27976	eval-f1score:0.66983
[325]	train-logloss:0.21324	train-f1score:0.73559	eval-logloss:0.28011	eval-f1score:0.66983
[350]	train-logloss:0.20979	train-f1score:0.74221	eval-logloss:0.28048	eval-f1score:0.66983
[375]	train-logloss:0.20759	train-f1score:0.74597	eval-logloss:0.28071	eval-f1score:0.66810
[400]	train-logloss:0.20498	train-f1score:0.75125	eval-logloss:0.28116	eval-f1score:0.66785
[425]	train-logloss:0.20154	train-f1score:0.75767	eval-logloss:0.28174	eval-f1score:0.66835
[445]	train-logloss:0.19936	train-f1score:0.76163	eval-logloss:0.28188	eval-f1sc

25:	learn: 0.5038325	test: 0.5024687	best: 0.5024687 (25)	total: 6.31s	remaining: 4m 44s
50:	learn: 0.4095408	test: 0.4074300	best: 0.4074300 (50)	total: 12.7s	remaining: 4m 47s
75:	learn: 0.3574025	test: 0.3550017	best: 0.3550017 (75)	total: 19.2s	remaining: 4m 44s
100:	learn: 0.3285133	test: 0.3261818	best: 0.3261818 (100)	total: 26s	remaining: 4m 42s
125:	learn: 0.3106320	test: 0.3086515	best: 0.3086515 (125)	total: 32.7s	remaining: 4m 38s
150:	learn: 0.2986332	test: 0.2970670	best: 0.2970670 (150)	total: 40.4s	remaining: 4m 40s
175:	learn: 0.2907224	test: 0.2894520	best: 0.2894520 (175)	total: 48.2s	remaining: 4m 40s
200:	learn: 0.2854808	test: 0.2847415	best: 0.2847415 (200)	total: 56.5s	remaining: 4m 40s
225:	learn: 0.2819504	test: 0.2815482	best: 0.2815482 (225)	total: 1m 4s	remaining: 4m 37s
250:	learn: 0.2792877	test: 0.2793523	best: 0.2793523 (250)	total: 1m 13s	remaining: 4m 36s
275:	learn: 0.2772961	test: 0.2778482	best: 0.2778482 (275)	total: 1m 21s	remaining: 4m 33s
300:	

1000:	learn: 0.2601864	test: 0.2796788	best: 0.2796788 (1000)	total: 5m 44s	remaining: 1m 8s
1025:	learn: 0.2599607	test: 0.2796315	best: 0.2796315 (1025)	total: 5m 54s	remaining: 1m
1050:	learn: 0.2596457	test: 0.2795650	best: 0.2795650 (1050)	total: 6m 5s	remaining: 51.8s
1075:	learn: 0.2593642	test: 0.2795469	best: 0.2795460 (1074)	total: 6m 16s	remaining: 43.3s
1100:	learn: 0.2591185	test: 0.2795311	best: 0.2795267 (1096)	total: 6m 27s	remaining: 34.8s
1125:	learn: 0.2588619	test: 0.2794750	best: 0.2794750 (1125)	total: 6m 38s	remaining: 26.2s
1150:	learn: 0.2586009	test: 0.2794613	best: 0.2794573 (1147)	total: 6m 49s	remaining: 17.4s
1175:	learn: 0.2583670	test: 0.2794342	best: 0.2794342 (1175)	total: 7m	remaining: 8.59s
1199:	learn: 0.2582426	test: 0.2794328	best: 0.2794322 (1193)	total: 7m 11s	remaining: 0us

bestTest = 0.279432168
bestIteration = 1193

Shrink model to first 1194 iterations.
--------------------------------------------------
catboost training fold 3
0:	learn: 0.

725:	learn: 0.2657655	test: 0.2665035	best: 0.2665035 (725)	total: 7m 10s	remaining: 4m 40s
750:	learn: 0.2653684	test: 0.2664256	best: 0.2664255 (748)	total: 7m 25s	remaining: 4m 26s
775:	learn: 0.2650262	test: 0.2663993	best: 0.2663993 (775)	total: 7m 40s	remaining: 4m 11s
800:	learn: 0.2646308	test: 0.2663184	best: 0.2663184 (800)	total: 7m 55s	remaining: 3m 56s
825:	learn: 0.2642923	test: 0.2662432	best: 0.2662432 (825)	total: 8m 10s	remaining: 3m 42s
850:	learn: 0.2639107	test: 0.2661918	best: 0.2661918 (850)	total: 8m 23s	remaining: 3m 26s
875:	learn: 0.2635619	test: 0.2661091	best: 0.2661091 (875)	total: 8m 37s	remaining: 3m 11s
900:	learn: 0.2632645	test: 0.2660675	best: 0.2660652 (899)	total: 8m 50s	remaining: 2m 55s
925:	learn: 0.2630559	test: 0.2660480	best: 0.2660447 (924)	total: 9m 2s	remaining: 2m 40s
950:	learn: 0.2626955	test: 0.2660255	best: 0.2660254 (949)	total: 9m 16s	remaining: 2m 25s
975:	learn: 0.2624185	test: 0.2660037	best: 0.2659981 (972)	total: 9m 30s	remaini

425:	learn: 0.2698182	test: 0.2817547	best: 0.2817547 (425)	total: 2m 50s	remaining: 5m 9s
450:	learn: 0.2690873	test: 0.2814949	best: 0.2814949 (450)	total: 3m	remaining: 5m
475:	learn: 0.2684703	test: 0.2812839	best: 0.2812839 (475)	total: 3m 11s	remaining: 4m 51s
500:	learn: 0.2678677	test: 0.2811056	best: 0.2811056 (500)	total: 3m 21s	remaining: 4m 41s
525:	learn: 0.2673000	test: 0.2810037	best: 0.2810026 (524)	total: 3m 30s	remaining: 4m 29s
550:	learn: 0.2667437	test: 0.2808307	best: 0.2808307 (550)	total: 3m 39s	remaining: 4m 18s
575:	learn: 0.2661682	test: 0.2806899	best: 0.2806899 (575)	total: 3m 47s	remaining: 4m 6s
600:	learn: 0.2656278	test: 0.2806421	best: 0.2806421 (600)	total: 3m 56s	remaining: 3m 55s
625:	learn: 0.2652042	test: 0.2805932	best: 0.2805888 (623)	total: 4m 4s	remaining: 3m 44s
650:	learn: 0.2647815	test: 0.2805400	best: 0.2805384 (647)	total: 4m 13s	remaining: 3m 33s
675:	learn: 0.2643910	test: 0.2805252	best: 0.2805252 (675)	total: 4m 22s	remaining: 3m 23s

In [138]:
def adaboost_inference(x_test):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(open(f'adaboost_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.pkl', 'rb'))
        # Predict
        pred = model.predict_proba(x_test)[:, 1]
        test_pred += pred
    return test_pred / CFG.n_folds

def nn_inference( x_test: pd.DataFrame):
    test_pred = np.zeros((x_test.shape[0],))
    for fold in range(CFG.n_folds):
        model = load_model(f'neuralnetwork_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.h5')
        # Predict
        pred = model.predict(x_test)
        pred = pred.flatten()  # 1 次元の配列に変換する
        test_pred += pred
    return test_pred / CFG.n_folds

def lightgbm_inference(x_test: pd.DataFrame):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(open(f'lightgbm_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.pkl', 'rb'))
        # Predict
        pred = model.predict(x_test)
        test_pred += pred
    return test_pred / CFG.n_folds
def xgboost_inference(x_test: pd.DataFrame):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(open(f'xgboost_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.pkl', 'rb'))
        # Predict
        pred = model.predict(xgb.DMatrix(x_test))
        test_pred += pred
    return test_pred / CFG.n_folds

def catboost_inference(x_test: pd.DataFrame):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(open(f'catboost_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.pkl', 'rb'))
        # Predict
        pred = model.predict_proba(x_test)[:, 1]
        test_pred += pred
    return test_pred / CFG.n_folds

def gradient_boosting_model_inference(method: str):
    x_test_lgb = test_df_lgb[features_lgb]
    x_test_ctb = test_df_ctb[features_ctb]
    x_test_nn = test_df_nn[features_nn]
    if method == 'adaboost':
        test_pred = adaboost_inference(x_test_ctb)
    if method == 'neuralnetwork':
        test_pred = nn_inference(x_test_nn)
    if method == 'lightgbm':
        test_pred = lightgbm_inference(x_test_ctb)
    if method == 'xgboost':
        test_pred = xgboost_inference(x_test_ctb)
    if method == 'catboost':
        test_pred = catboost_inference(x_test_ctb)
    return test_pred

def Predicting():
    output_df = test_df_lgb.copy()
    output_df['pred_prob'] = 0
    for method in CFG.METHOD_LIST:
        output_df[f'{method}_pred_prob'] = gradient_boosting_model_inference(method)
        output_df['pred_prob'] += CFG.model_weight_dict[method] * output_df[f'{method}_pred_prob']
    return output_df

In [139]:
test_df_lgb = Predicting()

1323/1323 [==============================] - 10s 7ms/step


In [140]:
#後処理の定義
def Postprocessing():
    train_df_lgb['pred_prob'] = 0
    for method in CFG.METHOD_LIST:
        oof_df = pd.read_csv(f'oof_{method}_seed{CFG.seed}_ver{CFG.VER}.csv')
        train_df_lgb['pred_prob'] += CFG.model_weight_dict[method] * oof_df[f'{method}_prediction']
    best_score = 0
    best_v = 0
    for v in tqdm(np.arange(1000) / 1000):
        score = f1_score(oof_df[CFG.target_col], train_df_lgb[f'pred_prob'] >= v, average='macro')
        if score > best_score:
            best_score = score
            best_v = v
    print(best_score, best_v)
    test_df_lgb['target'] = np.where(test_df_lgb['pred_prob'] >= best_v, 1, 0)
    return train_df_lgb, test_df_lgb

In [ ]:
#後処理
train_df, test_df = Postprocessing()

  0%|          | 0/1000 [00:00<?, ?it/s]

In [142]:
test_df[['target']].to_csv(f'seed{CFG.seed}_ver{CFG.VER}_{CFG.AUTHOR}_submission.csv', header=False)

In [143]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd

# OOF予測を基に新たな特徴量を作成
oof_features = np.zeros((train_df.shape[0], len(CFG.METHOD_LIST)))
for i, method in enumerate(CFG.METHOD_LIST):
    oof_df = pd.read_csv(f'oof_{method}_seed{CFG.seed}_ver{CFG.VER}.csv')
    oof_features[:, i] = oof_df[f'{method}_prediction']

# テストデータの予測を基に特徴量を作成
test_features = np.zeros((test_df.shape[0], len(CFG.METHOD_LIST)))
for i, method in enumerate(CFG.METHOD_LIST):
    test_features[:, i] = test_df[f'{method}_pred_prob']

# 特徴量の標準化
scaler = StandardScaler()
oof_features_scaled = scaler.fit_transform(oof_features)
test_features_scaled = scaler.transform(test_features)

# ロジスティック回帰モデルを学習
lr = LogisticRegression()
lr.fit(oof_features_scaled, train_df[CFG.target_col])

# 最適な閾値とその時のF1スコアを探索する関数
def find_best_threshold_and_score(y_true, y_pred_proba):
    best_threshold = 0
    best_score = 0
    for threshold in np.linspace(0, 1, 1001):  # 0.001刻みで閾値を変更
        score = f1_score(y_true, y_pred_proba >= threshold, average='macro')
        if score > best_score:
            best_score = score
            best_threshold = threshold
    return best_threshold, best_score

# 学習データに対する予測確率
train_pred_proba = lr.predict_proba(oof_features_scaled)[:, 1]

# 最適な閾値とスコアを求める
best_threshold, best_score = find_best_threshold_and_score(train_df[CFG.target_col], train_pred_proba)
print(f'Best Threshold: {best_threshold}, Best F1 Score: {best_score}')

# テストデータに対する最終予測
test_pred_proba = lr.predict_proba(test_features_scaled)[:, 1]
test_final_predictions = (test_pred_proba >= best_threshold).astype(int)

# 最終予渲結果をコンペ提出用のフォーマットでCSVファイルに出力
submission_df = pd.DataFrame({'Id': test_df.index, 'target': test_final_predictions}).reset_index(drop=True)
# ここで、インデックスの開始が42307であるため、その値から始めるように調整
submission_df['Id'] = submission_df.index + 42307

submission_df.to_csv(f'stacking_lr_submission_best_score{best_score:.4f}_seed{CFG.seed}_ver{CFG.VER}_{CFG.AUTHOR}.csv', header=False, index=False)


Best Threshold: 0.799, Best F1 Score: 0.687874789833567
